## Statisitical pattern classification

Hebbian-LMS and backpropagation neural networks for statistical pattern classification.Built using tensorflow and keras wrapper

In [1]:
import tensorflow as tf
import numpy as np
import math
import timeit
import matplotlib.pyplot as plt
import utils
import models
from PIL import Image

# This is a bit of magic to make matplotlib figures appear inline in the notebook
# rather than in a new window.
%matplotlib inline
plt.rcParams['figure.figsize'] = (20.0, 16.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
from keras.utils import np_utils

Using TensorFlow backend.


## Load training data

In [2]:
num_dim = 50 # number of dimensions
num_clusters = 100 # number of clusters
num_train_patterns = 100 # number of random patterns per cluster
num_test_patterns = 150 # number of random patterns per cluster

centroid_std = 1 # standard deviation of centroids
rho = 0.1;
patterns_std = rho*centroid_std*math.sqrt(2*num_dim); 

print (patterns_std)

def create_patterns(Ndim, Nclusters, Npatterns, centroids=None, centroid_std=1.0, patterns_std=1.0):
    '''
    Create |Npatterns| per cluster in |Ndim|-dimensional space. 
    Patterns are normally distributed around |centroids| with standard deviation |patterns_std|
    Centroids are normally distributed around the origin with standard deviation |centroid_std|
    
    Outputs:
    X_train: np array of shape (Npatterns*Nclusters, Ndim)
    Input patterns distributed around centroids. Xtrain[i, :] corresponds to the ith random pattern
    
    y_train: np array of shape (Npatterns*Nclusters,) 
    Centroid indeces. y_train[i] indicates the centroid which X_train[i, :] belongs to. 
    y_train must be converted to one-hot matrix
    
    centroids: np array of shape (Ncluster, Ndim)
    
    idx: np array of shape (Npatterns*Ncluster,)
    indeces used in shuffleling order of clusters and patterns
    '''
    if centroids is None:
        # create centroids
        centroids = centroid_std*np.random.randn(Nclusters, Ndim)
                    
    X_train = np.empty((0, Ndim))
    y_train = np.empty(0)
    for n in range(Nclusters):
        X_train = np.vstack((X_train, patterns_std*np.random.randn(Npatterns, Ndim) + centroids[n, :]))
        y_train = np.hstack((y_train, n*np.ones(Npatterns)))
      
    
    idx = np.arange(Npatterns*Nclusters)
    np.random.shuffle(idx)
    X_train = X_train[idx, :]
    y_train = y_train[idx]
    
    return X_train, y_train, centroids, idx
    
X_train, y_train, centroids, shuffle_idx = create_patterns(num_dim, num_clusters, num_train_patterns, centroid_std=centroid_std, patterns_std=patterns_std)
    
X_test, y_test, _, _ = create_patterns(num_dim, num_clusters, num_test_patterns, centroids=centroids, patterns_std=patterns_std)

1.0


In [9]:
num_hidden_layers = 3 # number of hidden layers 
num_neurons_hl = 150; # number of neurons per hidden layer
num_classes = num_clusters;

model = models.fully_connected((num_dim,), (num_hidden_layers, num_neurons_hl), num_classes)
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

y_train_categorical = np_utils.to_categorical(y_train, num_classes=num_classes);
y_test_categorical = np_utils.to_categorical(y_test, num_classes=num_classes);

batch_size = 100; #num_train_patterns*num_clusters
history = model.fit(X_train, y_train_categorical, batch_size=batch_size, epochs=10, validation_data=(X_test, y_test_categorical), verbose=True)

metrics = model.evaluate(X_test, y_test_categorical, verbose=True)


Train on 10000 samples, validate on 15000 samples
Epoch 1/10
10000/10000 [==============================] - 2s 196us/step - loss: 0.9028 - acc: 0.8637 - val_loss: 0.1010 - val_acc: 0.9918

In [3]:
num_hidden_layers = 3 # number of hidden layers 
num_neurons_hl = 150; # number of neurons per hidden layer
num_classes = num_clusters;

model = models.hlms_network((num_dim,), (num_hidden_layers, num_neurons_hl), num_classes)
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

y_train_categorical = np_utils.to_categorical(y_train, num_classes=num_classes);
y_test_categorical = np_utils.to_categorical(y_test, num_classes=num_classes);

batch_size = 100; #num_train_patterns*num_clusters
history = model.fit(X_train, y_train_categorical, batch_size=batch_size, epochs=10, validation_data=(X_test, y_test_categorical), verbose=True)

metrics = model.evaluate(X_test, y_test_categorical, verbose=True)

Train on 10000 samples, validate on 15000 samples
Epoch 1/10
10000/10000 [==============================] - 2s 219us/step - loss: 1.8585 - acc: 0.5750 - val_loss: 0.7577 - val_acc: 0.8034

In [4]:
print(model.get_weights())

[array([[ 0.26263043,  0.81393766,  1.56248105, ..., -0.95634615,
         0.54346436, -0.34454176],
       [ 0.91287696, -0.45150155,  0.29334232, ..., -0.93777364,
        -0.64602673,  0.3404389 ],
       [-1.30669248, -1.04471469, -1.75854039, ..., -0.75869691,
        -0.92716187,  0.02327711],
       ..., 
       [-0.96984327, -0.35335761, -0.94878197, ..., -2.69079137,
         0.71137148, -0.73940259],
       [-0.7859053 ,  0.07913699,  1.16846907, ..., -0.6410014 ,
        -1.11700332, -1.31899714],
       [-1.00331712,  0.14031693, -0.22916354, ..., -0.95075655,
        -1.07924318, -1.0362916 ]], dtype=float32), array([  2.63924122e-01,  -1.15818053e-01,   2.12222673e-02,
         3.79513919e-01,   3.11856018e-03,  -2.13967577e-01,
         1.85041264e-01,  -2.05413222e-01,  -4.62497845e-02,
         9.52184200e-02,   1.51365668e-01,  -1.48118390e-02,
        -1.80743024e-01,   5.62344752e-02,   7.32540488e-02,
        -6.28908128e-02,  -5.47496341e-02,   6.66225255e-01,
   

In [9]:
X_train, y_train, meanImage, ids = utils.load_training_data();

num_classes = len(ids);
crop = 0; 
X_train = utils.crop_image(X_train, crop, crop);
print("cropped X_train shape:", X_train.shape)
print("number of classes = ", num_classes)

X_train shape: (100000, 64, 64, 3)
y_train shape: (100000,)
cropped X_train shape: (100000, 64, 64, 3)
number of classes =  200


## Load validation data

In [11]:
X_val, y_val = utils.load_validation_data(ids, meanImage);
X_val = utils.crop_image(X_val, crop, crop);
print("cropped X_val shape:", X_val.shape)

X_val shape: (10000, 64, 64, 3)
y_val shape: (10000,)
cropped X_val shape: (10000, 64, 64, 3)


## Load test data

In [13]:
X_test, test_files = utils.load_test_data(meanImage);
X_test = utils.crop_image(X_test, crop, crop);
print("cropped X_test shape:", X_test.shape)

X_test shape: (10000, 64, 64, 3)
cropped X_test shape: (10000, 64, 64, 3)


## Train and evaluate model

In [15]:
# From https://gist.github.com/baraldilorenzo/07d7802847aaad0a35d3
model = models.VGG_16((64, 64, 3));

sgd = SGD(lr=0.03, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

y_train_categorical = np_utils.to_categorical(y_train, num_classes=200);
y_val_categorical = np_utils.to_categorical(y_val, num_classes=200);

history = model.fit(X_train, y_train_categorical, batch_size=200, epochs=20, validation_data=(X_val, y_val_categorical), verbose=True)
metrics = model.evaluate(X_val, y_val_categorical, verbose=True)


Train on 100000 samples, validate on 10000 samples
Epoch 1/20
100000/100000 [==============================] - 483s - loss: 5.1398 - acc: 0.0136 - val_loss: 5.1214 - val_acc: 0.0224
Epoch 2/20
100000/100000 [==============================] - 474s - loss: 4.7179 - acc: 0.0502 - val_loss: 4.7495 - val_acc: 0.0647
Epoch 3/20
100000/100000 [==============================] - 475s - loss: 4.2375 - acc: 0.1021 - val_loss: 4.3238 - val_acc: 0.1345
Epoch 4/20
100000/100000 [==============================] - 474s - loss: 3.8857 - acc: 0.1507 - val_loss: 4.0483 - val_acc: 0.1758
Epoch 5/20
100000/100000 [==============================] - 475s - loss: 3.6222 - acc: 0.1916 - val_loss: 3.8491 - val_acc: 0.2021
Epoch 6/20
100000/100000 [==============================] - 474s - loss: 3.4106 - acc: 0.2269 - val_loss: 3.6704 - val_acc: 0.2444
Epoch 7/20
100000/100000 [==============================] - 474s - loss: 3.2297 - acc: 0.2589 - val_loss: 3.6010 - val_acc: 0.2386
Epoch 8/20
100000/100000 [======

In [ ]:
from pylab import savefig

print(history.history.keys())

# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.ylabel('Accuracy', fontsize=20)
plt.xlabel('epoch', fontsize=20)
plt.legend(['Training', 'Validation'], loc='upper left')
plt.show()
savefig('accuracy.png', bbox_inches='tight')

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.ylabel('Loss', fontsize=20)
plt.xlabel('epoch', fontsize=20)
plt.legend(['Training', 'Validation'], loc='upper left')
plt.show()
savefig('loss.png', bbox_inches='tight')

model.save('milestone_model.h5')

utils.save_historty(history.historty, 'milestone_results.txt')


# Loading model
# from keras.models import load_model

# model.save('my_model.h5')  # creates a HDF5 file 'my_model.h5'
# del model  # deletes the existing model

# returns a compiled model
# identical to the previous one
# model = load_model('my_model.h5')

In [ ]:
## Save MNIST to matlab file